# 02a - Vertex AI - AutoML in GCP Console (no code)
Use Vertex AI from the GCP Console for a no-code approach building a custom model with AutoML and deploy it for predictions.

**Prerequisites:**
-  01 - BigQuery - Table Data Source

**Overview:**
-  Use the GCP Console > Vertex AI to
   -  Create a Dataset
      -  Pick data type and objective: Tabular, Regression/Classification
      -  Link to BigQuery Table for source data
   -  Create a Training Job
      -  Select the Dataset
      -  Objective = Classification with AutoML
   -  Evaluate Model
   -  Deploy Model to Endpoint
   -  Create a Batch Prediction Job 
      -  Use the Model and write predictions to BigQuery

**Resources:**
-  [AutoML Tabular Training Job With GCP Console](https://cloud.google.com/vertex-ai/docs/training/automl-console#tabular)

**Related Training:**
-  [Tutorial for AutoML Tabular](https://cloud.google.com/vertex-ai/docs/tutorials/tabular-automl)
-  todo


---
## Vertex AI - Conceptual Flow

<img src="architectures/slides/slide_07.png">

---
## Vertex AI - Workflow

<img src="architectures/slides/slide_08.png">

---
## Create Dataset (link to BigQuery table)

From the Console:
- Go to Vertex AI
- Selected `Datasets`
- Select `CREATE DATASET`

<img src="architectures/notebooks/02A_Screenshots/ds_1.png">

- Name the dataset `02a`
- Select `Tabular` and `Regression/classification`
- Click `Create`

<img src="architectures/notebooks/02A_Screenshots/ds_2.png">

- Under Select a data source pick `Select a table or view from BigQuery`
- Enter the BigQuery path (or browse) to the prepped table created in notebook 01
- Click `CONTINUE`

<img src="architectures/notebooks/02A_Screenshots/ds_3.png">

- The `ANALYZE` tab for the dataset will be displayed for review:

<img src="architectures/notebooks/02A_Screenshots/ds_4.png">

- Going back to the `Datasets` dashboard will display the registered dataset

<img src="architectures/notebooks/02A_Screenshots/ds_5.png">

---
## Train Model with AutoML

On The Vertex AI console, select `Training`:

<img src="architectures/notebooks/02A_Screenshots/train_1.png">

Next to `Training` (near the top), select `CREATE`
- For Dataset enter `02a`
- For Objective make sure `Classification` is selected
- Use `AutoML` for the method
- Click `CONTINUE`

<img src="architectures/notebooks/02A_Screenshots/train_2.png">

For `Model Details`:
- Keep the default `Model name` which appends a datetime to the end of the dataset name
- For `Target column` select the column to train predictions for
- Expand `ADVANCED OPTIONS`:
    - Select `Manual` for the Data split method
    - Select the `splits` variables that was created in Notebook 01
- Click `CONTINUE`

<img src="architectures/notebooks/02A_Screenshots/train_3.png">

For `Training options`:
- Click the `-` symbol next to any rows for variables that should be excluded from training, like the `transaction_id`
- Click `CONTINUE`

<img src="architectures/notebooks/02A_Screenshots/train_4.png">

For `Compute and pricing`:
- Enter a `Budget` of 1 node hour
    - A guide for choosing the right amount of time can be found [here](https://cloud.google.com/vertex-ai/docs/training/automl-console#tabular): 
- Make sure `Enable early stopping` is toggled on
- Click `START TRAINING`

<img src="architectures/notebooks/02A_Screenshots/train_5.png">

Return to the Vertex AI console `Training` Menu:
- Once the model completes training the name will be accompanied by a green check mark

<img src="architectures/notebooks/02A_Screenshots/train_6.png">

---
## Model: Evaluate, Select, Deploy

On the Vertex AI console, select `Models`

<img src="architectures/notebooks/02A_Screenshots/model_1.png">

Select the model that was just trained - starts with `02a`:
- This brings up the `EVALUATE` tab for the model

<img src="architectures/notebooks/02A_Screenshots/model_2.png">

Select the tab labeled `DEPLOY & TEST`:

<img src="architectures/notebooks/02A_Screenshots/model_3.png">

---
## Endpoint

While still on the Vetex AI `Models` section with the `DEPLOY & TEST` tab selected:
- select `DEPLOY TO ENDPOINT`

<img src="architectures/notebooks/02A_Screenshots/model_3.png">

In the `Deploy to endpoint` menus, complete `Define your endpoint`:
- For Endpoint name use `02a`
- keep defaults for location and Access
- Select `CONTINUE`

<img src="architectures/notebooks/02A_Screenshots/endpoint_1.png">

In the `Model settings` section:
- Traffic split should be 100
- minimum number of computes nodes is 1
- keep the remaining default values for max nodes, scaling, logging and explainability
- Select `CONTINUE`

<img src="architectures/notebooks/02A_Screenshots/endpoint_2.png">

In the `Model monitoring` section:
- Toggle `Enable model monitoring for this endpoint` on
    - for monitoring job use the name `02a`
    - use defaults for the other menue items
- Select `CONTINUE`

<img src="architectures/notebooks/02A_Screenshots/endpoint_3.png">

In the `Monitoring objectives` section:
- Select `Prediction Drift Detection` under Monitoring objective
- Select `DEPLOY`

<img src="architectures/notebooks/02A_Screenshots/endpoint_4.png">

Once the model is done being deployed to the endpoint, click the `Endpoints` section of Vertex AI:
- Select the endpoint that starts with `02a`
- Review the endpoint dashboard for the deployed model

<img src="architectures/notebooks/02A_Screenshots/endpoint_5.png">


---
## Batch

In the Verex AI console select the `Batch predictions` section:

<img src="architectures/notebooks/02A_Screenshots/batch_1.png">

Select `Create`:
- name the prediction `02a`
- for model name select the model that starts with `02a`
- for Select source, pick BigQuery table
- provide the location of the BigQuery source table
- for storage location pick output format of BigQuery
- provide the project for output in BigQuery
- select `Generate feature importance`
- select `Enable feature attributions for this model`
- select `CREATE`

<img src="architectures/notebooks/02A_Screenshots/batch_2.png">

Once the batch prediction job completes it will be listed with a green checkmark under `Batch Predictions`

<img src="architectures/notebooks/02A_Screenshots/batch_3.png">

Selecting the batch prediction job that starts with `02a` bring up the details of the prediction job

<img src="architectures/notebooks/02A_Screenshots/batch_4.png">

Select the linked BigQuery output table next to `Export location`:

<img src="architectures/notebooks/02A_Screenshots/batch_5.png">

---
## Prediction

inputs:

In [8]:
REGION = 'us-central1'
PROJECT_ID='statmike-mlops'
DATANAME = 'fraud'
NOTEBOOK = '02a'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters

packages:

In [9]:
from google.cloud import aiplatform

from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

clients:

In [10]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bigquery = bigquery.Client()

parameters:

In [11]:
DIR = f"temp/{NOTEBOOK}"

environment:

In [12]:
!rm -rf {DIR}
!mkdir -p {DIR}

### Prepare a record for prediction: instance and parameters lists

In [13]:
pred = bigquery.query(query = f"SELECT * FROM {DATANAME}.{DATANAME}_prepped WHERE splits='TEST' LIMIT 10").to_dataframe()

In [14]:
pred.head(4)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,75176,1.235603,0.041383,0.675286,0.836279,-0.675016,-0.657342,-0.154209,-0.067491,0.602617,...,0.088164,0.396205,0.324557,0.182930,-0.017115,0.014979,0.0,0,6f40e111-2131-4031-aef1-268b2daf02a3,TEST
1,112225,-0.285756,0.965688,2.147689,2.838137,1.104026,1.462921,-0.835272,-0.409875,-0.810586,...,-1.222639,0.044689,1.324343,-0.077670,0.128146,0.179651,0.0,0,c9bdeaf0-79d4-4faf-8aaa-14127a0e3513,TEST
2,113420,1.890283,0.241366,-0.165823,4.068924,-0.146807,0.140339,-0.258090,0.084852,-0.056567,...,0.097449,0.019180,0.060621,0.136349,-0.010745,-0.049814,0.0,0,611f4879-490d-4ab5-9e4f-c7944d6165eb,TEST
3,121910,-1.227268,1.555572,1.245848,4.071686,1.154573,2.058276,-1.179951,-2.211140,-2.248168,...,-0.380109,-1.485002,0.237729,0.524141,-0.037543,0.121962,0.0,0,3212444f-1412-49ae-adfa-a1a0ca54d8c5,TEST


In [20]:
#newob = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+[VAR_TARGET, 'splits'])]].to_dict(orient='records')[0]
newob = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+[VAR_TARGET])]].to_dict(orient='records')[0]
#newob

Need to understand the format of variables that the predictions expect.  AutoML may convert the type of some variables. The following cells retrieve the model from the endpoint and its schemata:

In [21]:
newob['Time'] = str(newob['Time'])

In [22]:
instances = [json_format.ParseDict(newob, Value())]
parameters = json_format.ParseDict({}, Value())

### Get Predictions: Python Client

In [23]:
endpoint = aiplatform.Endpoint.list(filter=f'display_name={NOTEBOOK}')[0]
endpoint.display_name

'02a'

In [24]:
prediction = endpoint.predict(instances=instances, parameters=parameters)
prediction

Prediction(predictions=[{'scores': [0.9995878338813782, 0.0004122101818211377], 'classes': ['0', '1']}], deployed_model_id='3774377127550386176', explanations=None)

In [25]:
prediction.predictions[0]['classes'][np.argmax(prediction.predictions[0]['scores'])]

'0'

### Get Predictions: REST

In [26]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": [newob]}))

In [27]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

{
  "predictions": [
    {
      "scores": [
        0.99958783388137817,
        0.00041221018182113772
      ],
      "classes": [
        "0",
        "1"
      ]
    }
  ],
  "deployedModelId": "3774377127550386176"
}


### Get Predictions: gcloud (CLI)

In [28]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[{'classes': ['0', '1'], 'scores': [0.9995878338813782, 0.0004122101818211377]}]


---
## Explanations
Interpretation Guide
- https://cloud.google.com/vertex-ai/docs/predictions/interpreting-results-automl#tabular

In [29]:
explanation = endpoint.explain(instances=instances, parameters=parameters)

In [30]:
explanation.predictions

[{'scores': [0.9995878338813782, 0.0004122101818211377],
  'classes': ['0', '1']}]

In [31]:
print("attribution:")
print("baseline output",explanation.explanations[0].attributions[0].baseline_output_value)
print("instance output",explanation.explanations[0].attributions[0].instance_output_value)
print("output_index",explanation.explanations[0].attributions[0].output_index)
print("output display value",explanation.explanations[0].attributions[0].output_display_name)
print("approximation error",explanation.explanations[0].attributions[0].approximation_error)

attribution:
baseline output 0.9996922612190247
instance output 0.9995878338813782
output_index [0]
output display value 0
approximation error 0.0049826675518549295


In [32]:
explanation.explanations[0].attributions[0]

baseline_output_value: 0.9996922612190247
instance_output_value: 0.9995878338813782
feature_attributions {
  struct_value {
    fields {
      key: "Amount"
      value {
        number_value: -1.25832027859158e-07
      }
    }
    fields {
      key: "Time"
      value {
        number_value: -6.443924374050564e-06
      }
    }
    fields {
      key: "V1"
      value {
        number_value: 6.466441684299045e-05
      }
    }
    fields {
      key: "V10"
      value {
        number_value: -6.000200907389323e-06
      }
    }
    fields {
      key: "V11"
      value {
        number_value: 3.337860107421875e-05
      }
    }
    fields {
      key: "V12"
      value {
        number_value: 6.145901150173611e-06
      }
    }
    fields {
      key: "V13"
      value {
        number_value: -9.304947323269314e-06
      }
    }
    fields {
      key: "V14"
      value {
        number_value: -2.620617548624675e-05
      }
    }
    fields {
      key: "V15"
      value {
        n

---
## Remove Resources
see notebook "XX - Cleanup"